In [101]:
import pandas as pd
from nltk import FreqDist
import pickle
import numpy as np
from gensim.models import Word2Vec

In [14]:
import numpy as np
import pandas as pd
import os
import requests
import re
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from cltk.stops.grc import STOPS as stopwords
from gensim.corpora import Dictionary
import unicodedata
import json

In [17]:
LAGT = pd.read_json("../data/large_data/LAGT_metadata.json")

In [18]:
LAGT.head(5)

,author_id,doc_id,GLAUx?,filename,subcorpus,author,title,date_avr,provenience,tlg_epithet,n_sentences,wordcount
0,tlg0001,tlg0001.tlg001,True,tlg0001.tlg001.perseus-grc2.xml,classical,Apollonius Rhodius,Argonautica,-2.5,pagan,Epici/-ae,3252,38822
1,tlg0003,tlg0003.tlg001,True,tlg0003.tlg001.perseus-grc2.xml,classical,Thucydides,The Peloponnesian War,-4.5,pagan,Historici/-ae,6068,150118
2,tlg0004,tlg0004.tlg001,False,tlg0004.tlg001.perseus-grc1.xml,roman,Diogenes Laertius,Lives of Eminent Philosophers,2.5,pagan,Biographi,10245,110763
3,tlg0005,tlg0005.tlg001,True,tlg0005.tlg001.perseus-grc1.xml,classical,Theocritus,Idylls,-3.0,pagan,Bucolici,1982,19200
4,tlg0005,tlg0005.tlg002,True,tlg0005.tlg002.perseus-grc1.xml,classical,Theocritus,Epigrams,-3.0,pagan,Bucolici,152,1734


In [33]:
subcorpora = ['archaic', 'classical', 'roman', 'christian', 'jewish']

# Read subcorpora from ngrams - developing a memory-friendly reader

In [19]:
ids_lines = pickle.load(open("../data/ids_lines.pickle", "rb"))

In [20]:
list(ids_lines.items())[:5]

[('tlg0001.tlg001', (0, 41193)),
 ('tlg0003.tlg001', (41193, 184616)),
 ('tlg0004.tlg001', (184616, 280036)),
 ('tlg0005.tlg001', (280036, 300693)),
 ('tlg0005.tlg002', (300693, 302641))]

In [21]:
subcorpus_ids = LAGT[LAGT["subcorpus"]=="archaic"]["doc_id"].tolist()
len(subcorpus_ids)

26

In [22]:
lines_list = [ ]
for id in subcorpus_ids:
    lines_list.extend([l for l in range(ids_lines[id][0], ids_lines[id][1])])
ngrams_n = len(lines_list)
ngrams_n

339423

In [23]:
sample_size=1000000
if sample_size != None:
    sample_inx =np.random.randint(0, ngrams_n, sample_size)
else:
    sample_inx =[n for n in range(0, ngrams_n)]

In [24]:
sample_lines = np.array(lines_list)[sample_inx]
len(sample_lines)

1000000

In [25]:
sample_lines[:5]

array([1977994, 8975012, 2081066, 2059048, 8972257])

In [26]:
test_ngrams = []
with open("../data/large_data/corpus_ngrams_bydocid.txt", "r") as f:
    lines = f.readlines()
    for li in sample_lines:
        line = lines[li]# for line index in subcorpus lineindexes
        if bool(line.split()):
            test_ngrams.append(line.split())

In [27]:
len(test_ngrams)

999639

# Sample corpora with memory-friendly reader

In [62]:
class NgramCorpusSample(object):
    def __init__(self, ids_list, ids_lines, fname, sample_size=None, sample_seed=1, bow=False, dct=None):
        self.ids_list = ids_list
        self.fname = fname
        self.bow  = bow
        self.dct = dct
        self.sample_size = sample_size
        self.ids_lines = ids_lines
        lines_list = [ ]
        for id in ids_list:
            lines_list.extend([l for l in range(ids_lines[id][0], ids_lines[id][1])])
        ngrams_n = len(lines_list)
        if sample_size != None:
            np.random.seed(sample_seed)
            sample_inx =np.random.randint(0, ngrams_n, sample_size)
        else:
            sample_inx =[n for n in range(0, ngrams_n)]
        sample_lines = np.array(lines_list)[sample_inx]
        self.len = len(sample_lines)
        self.sample_lines = sample_lines
    def __len__(self):
        #[el for el in self]
        return self.len
    def __iter__(self):
        with open(self.fname, "r") as f:
            lines = f.readlines()
            for li in self.sample_lines:
                line = lines[li] # for line index in subcorpus lineindexes
                if bool(line.split()):
                    if (self.bow) & (self.dct != None):
                        yield self.dct.doc2bow(line.split())
                    else:
                        yield line.split()

In [35]:
# test with classical

In [57]:
subcorpus_ids = LAGT[LAGT["subcorpus"]=="classical"]["doc_id"].tolist()
len(subcorpus_ids)

408

In [58]:
subcorpus_ngrams =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000)

In [59]:
len(subcorpus_ngrams)

1000000

In [60]:
len([doc for doc in subcorpus_ngrams])

991996

In [61]:
FreqDist([w for ng in subcorpus_ngrams for w in ng]).most_common()[:50]

[('εἰμί', 130478),
 ('οὗτος', 80423),
 ('αὐτός', 66886),
 ('τις', 41562),
 ('λέγω', 39489),
 ('ἔχω', 36401),
 ('γίγνομαι', 31603),
 ('πολύς', 30161),
 ('πᾶς', 28833),
 ('ἄλλος', 27519),
 ('ποιέω', 21280),
 ('λόγος', 16711),
 ('φημί', 15357),
 ('οὐδείς', 15301),
 ('μέγας', 15224),
 ('δέω', 13656),
 ('πόλις', 13129),
 ('τοιοῦτος', 12822),
 ('ἀγαθός', 12533),
 ('τίς', 11591),
 ('ἀνήρ', 11168),
 ('δοκέω', 11062),
 ('ἐκεῖνος', 11027),
 ('ἴσος', 10256),
 ('εἷς', 10053),
 ('λαμβάνω', 9730),
 ('ἕτερος', 9513),
 ('πρῶτος', 9509),
 ('ὅδε', 9352),
 ('ἄνθρωπος', 9279),
 ('ἕκαστος', 9000),
 ('δίδωμι', 8298),
 ('ὁράω', 8222),
 ('χρόνος', 7500),
 ('σῶμα', 7386),
 ('βούλομαι', 7347),
 ('οἶδα', 7172),
 ('θεός', 6910),
 ('ἔρχομαι', 6784),
 ('ἅπας', 6732),
 ('κακός', 6571),
 ('δύναμαι', 6538),
 ('ἀρχή', 6503),
 ('φαίνω', 6146),
 ('φύσις', 6125),
 ('μηδείς', 6113),
 ('Ἀθηναῖος', 6062),
 ('νόμος', 6020),
 ('καλός', 5845),
 ('δύο', 5824)]

In [43]:
with open("../data/large_data/subcorpora_vocabs.pickle", "rb") as f:
    subcorpora_vocabs = pickle.load(f)

In [93]:
subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=1)
subcorpus_ngrams_seed2 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=2)

In [94]:
subcorpora_vocabs["classical"]["word_freqs"]

[('εἰμί', 95760),
 ('οὗτος', 61901),
 ('αὐτός', 46507),
 ('λέγω', 31322),
 ('τις', 28324),
 ('ἔχω', 26149),
 ('γίγνομαι', 23004),
 ('πολύς', 21356),
 ('πᾶς', 20390),
 ('ἄλλος', 19388),
 ('ποιέω', 15520),
 ('φημί', 12569),
 ('λόγος', 12015),
 ('οὐδείς', 11032),
 ('τίς', 10804),
 ('μέγας', 10469),
 ('δέω', 10068),
 ('τοιοῦτος', 9389),
 ('πόλις', 8960),
 ('ἀγαθός', 8934),
 ('δοκέω', 8503),
 ('ἐκεῖνος', 7996),
 ('ἀνήρ', 7966),
 ('ἴσος', 7557),
 ('ὅδε', 7240),
 ('εἷς', 7158),
 ('λαμβάνω', 7042),
 ('πρῶτος', 7005),
 ('ἕτερος', 6931),
 ('ἄνθρωπος', 6476),
 ('ὁράω', 6364),
 ('ἕκαστος', 6126),
 ('δίδωμι', 6087),
 ('οἶδα', 5493),
 ('βούλομαι', 5420),
 ('σῶμα', 5205),
 ('χρόνος', 5191),
 ('θεός', 5050),
 ('ἔρχομαι', 5010),
 ('φαίνω', 4807),
 ('κακός', 4788),
 ('ἅπας', 4761),
 ('ἀρχή', 4633),
 ('δύναμαι', 4584),
 ('φύσις', 4459),
 ('νόμος', 4243),
 ('οἴομαι', 4242),
 ('δύο', 4185),
 ('καλός', 4178),
 ('Ἀθηναῖος', 4171),
 ('ὑπάρχω', 4141),
 ('μηδείς', 4110),
 ('πράσσω', 4103),
 ('γῆ', 3934),
 ('χρά

In [95]:
freqs_comparison = pd.DataFrame(
    [dict(subcorpora_vocabs["classical"]["word_freqs"][:5000]),
    dict(FreqDist([w for ng in subcorpus_ngrams_seed1 for w in ng]).most_common()[:5000]),
    dict(FreqDist([w for ng in subcorpus_ngrams_seed2 for w in ng]).most_common()[:5000])]
).T


In [96]:
freqs_comparison.dropna().corr()

,0,1,2
0,1.000000,0.999110,0.999012
1,0.999110,1.000000,0.999932
2,0.999012,0.999932,1.000000


In [97]:
len(freqs_comparison)

5364

In [98]:
freqs_comparison[5000:5100]

,0,1,2
Μαίανδρος,NaN,76.0,NaN
βαλανεῖον,NaN,66.0,NaN
ἐπαείδω,NaN,64.0,54.0
παμπληθής,NaN,64.0,51.0
ἐνέχυρον,NaN,63.0,NaN
...,...,...,...
ἀριστεῖα,NaN,50.0,64.0
πύρινος,NaN,50.0,NaN
Στρατοκλῆς,NaN,50.0,NaN
Μυκηναῖος,NaN,50.0,50.0


In [99]:
freqs_comparison[:100].corr()

,0,1,2
0,1.000000,0.998822,0.998680
1,0.998822,1.000000,0.999954
2,0.998680,0.999954,1.000000


In [100]:
np.log2(freqs_comparison).corr()

,0,1,2
0,1.000000,0.995327,0.995393
1,0.995327,1.000000,0.994106
2,0.995393,0.994106,1.000000


In [106]:
# let's build two models based on the same subcorpus but sampled with different seed

In [107]:
%%time
sub = "classical"
subcorpus_ids = LAGT[LAGT["subcorpus"]=="classical"]["doc_id"].tolist()
subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=1)
model_seed1 = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
model_seed1.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
model_seed1.train(subcorpus_ngrams, total_examples=len(subcorpus_ngrams_seed1), epochs=model_seed1.epochs)
vectors_seed1 = model_seed1.wv

subcorpus_ngrams_seed2 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=2)
model_seed2 = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
model_seed2.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
model_seed2.train(subcorpus_ngrams, total_examples=len(subcorpus_ngrams_seed2), epochs=model_seed1.epochs)
vectors_seed2 = model_seed2.wv

CPU times: user 1min 29s, sys: 1min 17s, total: 2min 47s
Wall time: 3min 3s


In [110]:
vectors_seed1.most_similar("θεός")

[('δαίμων', 0.6374392509460449),
 ('εὐχή', 0.5827338695526123),
 ('εὔχομαι', 0.5637454986572266),
 ('θυσία', 0.560525119304657),
 ('δαιμόνιον', 0.5602738261222839),
 ('προσκυνέω', 0.5185097455978394),
 ('εὐσεβής', 0.5075002908706665),
 ('οἰωνός', 0.5066176056861877),
 ('σεμνός', 0.5062194466590881),
 ('ἀθάνατος', 0.5041354298591614)]

In [111]:
vectors_seed2.most_similar("θεός")

[('δαίμων', 0.6391631960868835),
 ('εὐχή', 0.5830387473106384),
 ('δαιμόνιον', 0.5722740292549133),
 ('θυσία', 0.5643562078475952),
 ('ἀθάνατος', 0.5316399931907654),
 ('ἀτιμάζω', 0.5305225253105164),
 ('οἰωνός', 0.5217851400375366),
 ('εὐσεβής', 0.5193997025489807),
 ('θνητός', 0.5141244530677795),
 ('εὔχομαι', 0.5113592147827148)]

In [ ]:
for sub in subcorpora:
    #do the stuff above
    print(sub)